In [129]:
import cv2
import time
import math
import json
import argparse
import numpy as np

In [148]:
path = '../../data/lena.jpg'
src = cv2.imread(path)
window_name = 'Image'
num_times = 100

def testFunc(func_name, func, json, *args):
    start_time = time.time()
    for i in range(num_times):
        image = func( *args)
    end_time = time.time()
    time_taken = (end_time - start_time) * 1000
    json[func_name] = {'ms': time_taken}
    print(func_name + " time for " + str(num_times) + " runs in ms:", time_taken)
        
def run():
    out = {}
    # transform
    testFunc('resize', cv2.resize, out,src, (100, 100))
    testFunc('translation', cv2.warpAffine, out,src, cv2.getRotationMatrix2D((50, 50), 0, 1), (100, 100))
    testFunc('rotate 90', cv2.rotate, out,src, cv2.ROTATE_90_CLOCKWISE)
    testFunc('affine transformation', cv2.warpAffine, out,src, np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float32), (100, 100))
    testFunc('perspective transformation', cv2.warpPerspective, out,src, np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype=np.float32), (100, 100))

    # colorspace
    testFunc('grayscale', cv2.cvtColor, out,src, cv2.COLOR_BGR2GRAY)
    testFunc('inRange', cv2.inRange, out,src, (0, 0, 0), (255, 255, 255))
    testFunc('hsv', cv2.cvtColor, out,src, cv2.COLOR_BGR2HSV)

    # threshold
    testFunc('threshold', cv2.threshold, out,src, 177, 200, cv2.THRESH_BINARY)
    src2 = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
    testFunc('adaptive threshold', cv2.adaptiveThreshold, out,src2, 200, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)

    # smoothening
    kernel = np.ones((5, 5), np.float32)/25
    testFunc('2d convolution', cv2.filter2D, out, src, -1, kernel)
    testFunc('mean blur', cv2.blur, out,src, (25,25))
    testFunc('gaussian blur', cv2.GaussianBlur, out,src, (25, 25), 0)
    testFunc('median blur', cv2.medianBlur, out,src, 25)
    testFunc('bilateral filter', cv2.bilateralFilter, out,src, 9, 75, 75)

   # Morphological Transformations
    testFunc('erosion', cv2.erode, out,src, np.ones((5, 5), np.uint8), 1)
    testFunc('dilation', cv2.dilate, out,src, np.ones((5, 5), np.uint8), 1)
    testFunc('opening', cv2.morphologyEx, out,src, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    testFunc('closing', cv2.morphologyEx, out,src, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
    testFunc('gradient', cv2.morphologyEx, out,src, cv2.MORPH_GRADIENT, np.ones((5, 5), np.uint8))
    testFunc('top hat', cv2.morphologyEx, out,src, cv2.MORPH_TOPHAT, np.ones((5, 5), np.uint8))
    testFunc('black hat', cv2.morphologyEx, out,src, cv2.MORPH_BLACKHAT, np.ones((5, 5), np.uint8))
    
    # gradients
    testFunc('sobel', cv2.Sobel, out,src, cv2.CV_64F, 1, 0, 5)
    testFunc('scharr', cv2.Scharr, out,src, cv2.CV_64F, 1, 0)
    testFunc('laplacian', cv2.Laplacian, out,src, cv2.CV_64F)

    # edge detection
    testFunc('canny', cv2.Canny, out,src, 100, 200)

   # contours
    def findCountors():
        imgray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, 0)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        return contours

    # countours
    testFunc('contours', findCountors, out)
    
    # histogram
    def histogram():
        color = ('b','g','r')
        for i,col in enumerate(color):
            histr = cv.calcHist([src],[i],None,[256],[0,256])
    testFunc('histogram', histogram, out)
    testFunc('equalize histogram', cv2.equalizeHist, out, src2)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    testFunc('clahe', clahe.apply, out, src2)

    face_cascade = cv.CascadeClassifier('../../data/haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier('../../data/haarcascade_eye.xml')
    def haarcascade():
        gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            center = (x + w//2, y + h//2)
            frame = cv.ellipse(src, center, (w//2, h//2), 0, 0, 360, 255, 4)
            faceROI = gray[y:y+h,x:x+w]
            eyes = eye_cascade.detectMultiScale(faceROI)
            for (x2,y2,w2,h2) in eyes:
                eye_center = (x + x2 + w2//2, y + y2 + h2//2)
                radius = int(round((w2 + h2)*0.25))
                frame = cv.circle(frame, eye_center, radius, (255, 0, 0 ), 4)


    # haar cascade
    testFunc('haar cascade face', haarcascade, out)

    with open("out.json", "w") as file:
        json.dump(out, file)

In [149]:
run()

resize time for 100 runs in ms: 16.95394515991211
translation time for 100 runs in ms: 8.63790512084961
rotate 90 time for 100 runs in ms: 31.78095817565918
affine transformation time for 100 runs in ms: 7.272958755493164
perspective transformation time for 100 runs in ms: 12.696981430053711
grayscale time for 100 runs in ms: 6.011962890625
inRange time for 100 runs in ms: 22.01700210571289
hsv time for 100 runs in ms: 11.044979095458984
threshold time for 100 runs in ms: 2.971172332763672
adaptive threshold time for 100 runs in ms: 55.15003204345703
2d convolution time for 100 runs in ms: 141.2680149078369
mean blur time for 100 runs in ms: 60.423851013183594
gaussian blur time for 100 runs in ms: 121.57273292541504
median blur time for 100 runs in ms: 4167.103052139282
bilateral filter time for 100 runs in ms: 1598.3657836914062
erosion time for 100 runs in ms: 19.739151000976562
dilation time for 100 runs in ms: 24.202823638916016
opening time for 100 runs in ms: 34.46698188781738
c